# Autoencoders

- In classical machine learning, we introduced PCA as a dimensionality reduction algorithm, in deep learning auto-encoder has the same purpose as PCA does for tabular data.
- "Autoencoding" is a data compression and decompression algorithm where the compression and decompression functions are implemented with neural networks. 
- Two interesting practical applications of autoencoders are data denoising (which we feature later in this post), and dimensionality reduction

![Autoencoder](../Images/auto_encoder.png)

## Activity: Create an autoencoder with keras based on the image above

In [34]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras import regularizers
import numpy as np

encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

input_img = Input(shape=(784,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input_img, decoded)

autoencoder.compile(optimizer='adadelta', loss='mse')


# prepare our input data. We're using MNIST digits, and we're discarding the labels
# since we're only interested in encoding/decoding the input images
(x_train, _), (x_test, _) = mnist.load_data()

# normalize all values between 0 and 1 and we will flatten the 28x28 images into vectors of size 784
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Reshape x_train and x_test
x_train = np.reshape(x_train,[-1, 28*28])
x_test = np.reshape(x_test,[-1, 28*28])

# train our autoencoder for 50 epochs
autoencoder.fit(x_train, x_train, epochs=50, batch_size=256, shuffle=True, validation_data=(x_test, x_test), verbose=0)

In [35]:
# We should create a separate encoder model this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# As well as the decoder model
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)